In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam
import os.path
import json
from keras.models import model_from_json

# For reproducibility - splitting train and test sets
seed = 123
np.random.seed(seed)


# Load data from Excel sheets

dataset1 = pd.read_excel('Simulink_Data_25.xlsx')
dataset2 = pd.read_excel('Simulink_Data_25_1.xlsx')
dataset3 = pd.read_excel('Simulink_Data_50.xlsx')
dataset4 = pd.read_excel('Simulink_Data_50_1.xlsx')
dataset5 = pd.read_excel('Simulink_Data_75.xlsx')
dataset6 = pd.read_excel('Simulink_Data_75_1.xlsx')
dataset7 = pd.read_excel('Simulink_Data_100.xlsx')
dataset8 = pd.read_excel('Simulink_Data_100_1.xlsx')
#Combine datasets into one single data file
frames=[dataset1,dataset2,dataset3,dataset4,dataset5,dataset6,dataset7,dataset8]
dataset = pd.concat(frames)

#  Shuffle Data
dataset = dataset.sample(frac=1).reset_index(drop=True)
X_complete=dataset.drop('class',axis=1)
y_complete=dataset['class']

print(type(X_complete))
print("Unnormalized Data", "\n", X_complete[:5], "\n")

<class 'pandas.core.frame.DataFrame'>
Unnormalized Data 
     CO2_Zone_1   CO2_Zone_2   CO2_Zone_3   CO2_Zone_4   CO2_Zone_5  \
0   582.242991   815.371584   573.770973   736.007364   533.105710   
1  1285.206632  1262.129709  1459.823383  1430.411951  1335.115158   
2   483.022072   635.978451   477.837200   607.979247   452.949922   
3  1666.038626  1700.412835  1666.040031  1683.226148  1674.632006   
4  1263.113447  1576.165384  1388.204880  1278.984458  1320.229164   

    CO2_Zone_6  pas_cnt  
0   690.753919       47  
1  1225.653504       93  
2   564.857383       47  
3  1694.683670      186  
4  1377.015714      140   



In [19]:



# Feature scaling according to training set data 
col=list(X_complete.columns.values)

for i in col:
    avg=X_complete[str(i)].mean()
    sd=X_complete[str(i)].std()
    X_complete[str(i)]=X_complete[str(i)].apply(lambda X:(X-avg)/(sd))

print("Normalized Data\n", X_complete[:5], "\n")


# covert to array for processing
X_complete=X_complete.values

#One hot encoding
y_complete = pd.get_dummies(y_complete).values



Normalized Data
    CO2_Zone_1  CO2_Zone_2  CO2_Zone_3  CO2_Zone_4  CO2_Zone_5  CO2_Zone_6  \
0   -1.088624   -0.950637   -1.235218   -1.170648   -1.368075   -1.208656   
1    0.274130   -0.022274    0.525111    0.329783    0.224104   -0.066552   
2   -1.280972   -1.323416   -1.425811   -1.447284   -1.527203   -1.477467   
3    1.012405    0.888478    0.934804    0.876050    0.898126    0.934910   
4    0.231301    0.630292    0.382826    0.002587    0.194552    0.256633   

    pas_cnt  
0 -1.339693  
1 -0.452990  
2 -1.339693  
3  1.339693  
4  0.452990   



In [33]:
# Creating a Train and a Test Dataset
X_train, X_test, y_train, y_test = train_test_split(X_complete, y_complete, test_size=0.3, random_state=seed)


# Define Neural Network model layers
model = Sequential()
model.add(Dense(20, input_dim=7, activation='relu'))
model.add(Dense(20, input_dim=7, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile model
model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])


#ttt=1

if os.path.isfile('#MLP_CO2_2019_weights.h5'):

    # Model reconstruction from JSON file
    json_file = open('MLP_CO2_2019_architecture.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    
    # Load weights into the new model
    model.load_weights('MLP_CO2_2019_weights.h5')
    print("Model weights loaded from saved model data.")

    model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
else:
#if ttt==1:
    print("Model weights data not found. Model will be fit on training set now.")

    # Fit model on training data - try to replicate the normal input
    model.fit(X_train,y_train,epochs=10,batch_size=256,verbose=1,validation_data=(X_test,y_test))
    
    # Save parameters to JSON file
    model_json = model.to_json()
    with open("MLP_2019_architecture.json", "w") as json_file:
        json_file.write(model_json)

    # Save model weights to file
    model.save_weights('MLP_CO2_2019_weights.h5')


model.summary()





Model weights data not found. Model will be fit on training set now.
Train on 60485 samples, validate on 25923 samples
Epoch 1/10
60485/60485 [==============================] - 2s 25us/step - loss: 0.1920 - acc: 0.9223 - val_loss: 0.1389 - val_acc: 0.9480
Epoch 2/10
60485/60485 [==============================] - 1s 12us/step - loss: 0.1405 - acc: 0.9438 - val_loss: 0.1508 - val_acc: 0.9284
Epoch 3/10
60485/60485 [==============================] - 1s 11us/step - loss: 0.1359 - acc: 0.9464 - val_loss: 0.1435 - val_acc: 0.9478
Epoch 4/10
60485/60485 [==============================] - 1s 11us/step - loss: 0.1311 - acc: 0.9485 - val_loss: 0.1229 - val_acc: 0.9523
Epoch 5/10
60485/60485 [==============================] - 1s 12us/step - loss: 0.1318 - acc: 0.9498 - val_loss: 0.1203 - val_acc: 0.9516
Epoch 6/10
60485/60485 [==============================] - 1s 11us/step - loss: 0.1259 - acc: 0.9520 - val_loss: 0.1218 - val_acc: 0.9539
Epoch 7/10
60485/60485 [==============================] - 1

In [35]:
# Model predictions for test set
y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)


print(y_test_class,y_pred_class)

# Evaluate model on test data
score = model.evaluate(X_test, y_test, batch_size=128,verbose=1)
print(score)

# Compute stats on the test set and Output all results
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))


[1 1 0 ... 0 1 1] [1 1 0 ... 0 1 1]
25923/25923 [==============================] - 0s 6us/step
[0.1068155754390485, 0.9590710952308348]
             precision    recall  f1-score   support

          0       0.96      0.98      0.97     20202
          1       0.94      0.87      0.90      5721

avg / total       0.96      0.96      0.96     25923

[[19886   316]
 [  745  4976]]


In [31]:
 model.load_weights('MLP_CO2_2019_weights.h5')

In [30]:
 model.save_weights('MLP_CO2_2019_weights.h5')
